# Credit Risk Ensemble Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)

df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,9858.327514,7.314789,rent,49433.310055,0.197167,4.0,1.0,19433.310055,low_risk
1,13038.876833,8.666523,mortgage,62155.507334,0.260778,7.0,2.0,32155.507334,low_risk
2,13059.502773,8.675289,own,62238.011090,0.261190,7.0,2.0,32238.011090,low_risk
3,9469.446028,7.149515,mortgage,47877.784110,0.189389,4.0,1.0,17877.784110,low_risk
4,8948.318632,6.928035,rent,45793.274526,0.178966,4.0,1.0,15793.274526,low_risk


In [6]:
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)

df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [7]:
# Create the LabelEncoder instance
le = LabelEncoder()

# Fitting and encoding the columns with the LabelEncoder
# homeowner column
le.fit(df["homeowner"])
df["homeowner"] = le.transform(df["homeowner"])

df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,1,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,1,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,2,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,1,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,0,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [8]:
# Create our target
y = df["loan_status"]

# Create our features
X = df.drop(columns="loan_status")


In [9]:
X.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.606144,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,0.667811,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [6]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000,55000.000000
mean,11980.701981,8.216798,57922.807923,0.239614,6.281636,1.360855,27922.807923,0.503309,0.396982,0.099709
std,2428.659146,1.032180,9714.636584,0.048573,2.203214,0.590277,9714.636584,0.499994,0.489277,0.299614
min,5000.000000,5.250000,30000.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10439.956986,7.561982,51759.827942,0.208799,5.000000,1.000000,21759.827942,0.000000,0.000000,0.000000
50%,11609.670503,8.059110,56438.682013,0.232193,6.000000,1.000000,26438.682013,1.000000,0.000000,0.000000
75%,12902.612993,8.608611,61610.451970,0.258052,7.000000,2.000000,31610.451970,1.000000,1.000000,0.000000
max,22769.252782,12.801932,101077.011129,0.455385,16.000000,4.000000,71077.011129,1.000000,1.000000,1.000000


In [10]:
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [7]:
# Check the balance of our target values
# YOUR CODE HERE

low_risk     50000
high_risk     5000
Name: loan_status, dtype: int64

In [8]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)
X_train.shape

(58152, 8)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [17]:
# Resample the training data with the RandomOversampler
from imblearn.ensemble import BalancedRandomForestClassifier
model = BalancedRandomForestClassifier(random_state=1)
model.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [18]:
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9927455343210623

In [10]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

0.9927988250218349

In [15]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  615,     4],
       [  151, 18614]], dtype=int64)

In [11]:
# Display the confusion matrix
# YOUR CODE HERE

array([[  615,     4],
       [  149, 18616]])

In [16]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

pre       rec       spe        f1       geo       iba       sup

  high_risk       0.80      0.99      0.99      0.89      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



In [12]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.80      0.99      0.99      0.89      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



In [13]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE

loan_size: (0.21788480603139448)
interest_rate: (0.19595880072166166)
borrower_income: (0.1770964986362893)
debt_to_income: (0.13872579131432708)
num_of_accounts: (0.12149751059258558)
derogatory_marks: (0.11796397199368643)
total_debt: (0.028820980671490287)
homeowner_mortgage: (0.0008975938107589897)
homeowner_own: (0.0008326989735687767)
homeowner_rent: (0.00032134725423733495)


### Easy Ensemble AdaBoost Classifier

In [20]:
from imblearn.ensemble import EasyEnsembleClassifier
model = EasyEnsembleClassifier(n_estimators=100,random_state=1)
model.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [14]:
# Train the Classifier


EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [21]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.993171859927244

In [22]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  615,     4],
       [  135, 18630]], dtype=int64)

In [23]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

pre       rec       spe        f1       geo       iba       sup

  high_risk       0.82      0.99      0.99      0.90      0.99      0.99       619
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384

